# [o3]- Proyecto Ozono - Predictor_v0

# [0] - Inicialización

In [1]:
import findspark
findspark.init('/home/rulicering/BigData/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd
from pyspark.sql.types import StructField,StringType,IntegerType,StructType,FloatType
import re as reg
import numpy as np
import datetime

#MlLib
#from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

#Aux
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [2]:
spark = SparkSession.builder.appName('predictor').getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

# [1] Datos

## [1.0] - Carga de ficheros (Datos, Predicción clima,  Calendario)
    Se ejecuta el predictor a las 2 am del dia siguiente

In [3]:
ayer = (datetime.date.today() + datetime.timedelta(days = -1)).strftime("%Y-%m-%d")
hoy = datetime.date.today().strftime("%Y-%m-%d")

In [4]:
df_datos = spark.read.csv('/home/rulicering/Datos_Proyecto_Ozono/Procesado/Dato_Final/Datos.csv',inferSchema= True,header=True)
df_clima_prediccion = spark.read.csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Clima/Clima_Prediccion-"+ hoy + ".csv",inferSchema= True,header=True)
df_calendario = spark.read.csv('/home/rulicering/Datos_Proyecto_Ozono/Procesado/Calendario/Calendario_2001-2020.csv',inferSchema= True,header=True)

In [5]:
df_datos = df_datos.drop("_c0")
df_clima_prediccion = df_clima_prediccion.drop("_c0")
df_calendario = df_calendario.drop("_c0")

In [6]:
magnitudes= df_datos.columns[8:]
magnitudes_clima = df_datos.columns[-5:]
magnitudes_aire = df_datos.columns[8:-5]

In [7]:
dic_clima = { "VIENTO":"81",
                 "DIRECCION": "82",
                 "TEMPERATURA": "83",
                 "PRESION": "87",
                 "LLUVIA":"89"
}

## [1.1] - Datos para prediccion - Prediccion clima + Calendario + Estaciones

In [8]:
ayer = (datetime.date.today() + datetime.timedelta(days = -1)).strftime("%Y%m%d")
hoy = datetime.date.today().strftime("%Y%m%d")

In [9]:
df_estaciones_aire = df_datos.filter(df_datos["FECHA"]== ayer).select("CODIGO_CORTO")

In [10]:
cod_estaciones_aire = [elem[0] for elem in df_estaciones_aire.collect()]

In [11]:
cod_estaciones_aire.sort()

In [12]:
df_hoy = df_calendario.filter(df_calendario["FECHA"]== hoy)

In [13]:
#Calendario + Magnitudes aire a null
for magnitud in magnitudes_aire:
    df_hoy = df_hoy.withColumn(magnitud,F.lit(None))

In [14]:
#Calendario + Prediccion clima
df_clima_hoy = df_hoy.join(df_clima_prediccion,on= "FECHA")

In [15]:
#Estaciones cross datos clima y calendario
df_datos_hoy = df_estaciones_aire.crossJoin(df_clima_hoy)

In [16]:
cols = df_datos_hoy.columns
cols = cols[0:1] + cols[19:22]+ cols[1:5]+ cols[5:19] + cols[22:]

In [17]:
df_datos_hoy = df_datos_hoy.select(cols)

### [1.1.0] - Probabilidad lluvia -> Prediccion lluvia m/l2 + Presion dia anterior
    Si la probabilidad es > 50%:
        se hace la media por estacion del historial de precipitaciones
        cogiendo datos de +-10 días al dia de hoy de cada año anterior

In [18]:
def probabilidad_a_lluvia_presion_ayer_aire_a_null(df_datos,df_datos_hoy):
    mes_dia_min = (datetime.date.today() +  datetime.timedelta(days = -10)).strftime("%m%d")
    mes_dia_max = (datetime.date.today() +  datetime.timedelta(days = 10)).strftime("%m%d")
    df_historial = df_datos.filter((df_datos["FECHA"]%1000 >= mes_dia_min) & (df_datos["FECHA"]%1000 <= mes_dia_max))
    df_datos_hoy = df_datos_hoy.drop('%' + dic_clima["LLUVIA"])
    l_df = []
    for estacion in cod_estaciones_aire:
        df_datos_hoy_estacion = df_datos_hoy.filter(df_datos_hoy["CODIGO_CORTO"]==estacion)
        aux = df_historial.filter(df_historial["CODIGO_CORTO"]== estacion).select("FECHA",dic_clima["PRESION"],dic_clima["LLUVIA"]).na.drop()
        #Precipitacions
        prob_lluvia_hoy = df_clima_prediccion.select("%" +dic_clima["LLUVIA"]).collect()[0][0]
        prec = 0
        if(float(prob_lluvia_hoy) > 50):
            try:
                prec = aux.filter(aux[dic_clima["LLUVIA"]] >0).select(dic_clima["LLUVIA"]).groupBy().mean().collect()[0][0]
            except:
                print("[WARN]: No hay lluvias historicas en ese rango de fechas")
        df_datos_hoy_estacion = df_datos_hoy_estacion.withColumn(dic_clima["LLUVIA"],F.lit(prec))
            
        #Presion
        ayer = (datetime.date.today() + datetime.timedelta(days= -1)).strftime("%Y%m%d")
        presion_ayer = aux.filter(aux["FECHA"]==ayer).select(dic_clima["PRESION"]).collect()[0][0]
        df_datos_hoy_estacion = df_datos_hoy_estacion.withColumn(dic_clima["PRESION"],F.lit(presion_ayer))

        l_df.append(df_datos_hoy_estacion)
        
    df_datos_hoy = l_df[0]
    for i in range(1,len(l_df)):
        df_datos_hoy = df_datos_hoy.union(l_df[i])
    return df_datos_hoy

In [19]:
df_datos_hoy = probabilidad_a_lluvia_presion_ayer_aire_a_null(df_datos,df_datos_hoy)

## [1.2] - Union Datos + Datos hoy

In [20]:
df_datos= df_datos.union(df_datos_hoy)

## [1.3] - Dar cada fila de datos +  contaminacion ayer

In [21]:
ventana = Window.partitionBy("CODIGO_CORTO").orderBy("FECHA")

In [22]:
for magnitud in magnitudes_aire:
    df_datos = df_datos.withColumn("A_%s"%magnitud, F.lag(magnitud,1,None).over(ventana))

## [1.4] - Tipos
    
    CODIGO_CORTO -> INTEGER
    ANO -> INTEGER
    MES -> INTEGER
    DIA -> INTEGER
    FECHA -> INTEGER
    DIASEMANA -> INTEGER
    TIPODIA -> INTEGER
    CONFINAMIENTO -> INTEGER
    MEDICIONES -> DOUBLE

# [2] - PREDICCIONES
    
    Se hace 1 a 1 para cada magnitud de contaminación
    

In [23]:
cols_comunes = df_datos.columns[0:8] + magnitudes_clima

## [2.1] - GBT

In [24]:
l_predicciones = []
for magnitud in magnitudes_aire:
    print("="*20, magnitud, "="*20)
    cols_comunes = df_datos.columns[0:8] + magnitudes_clima
    cols_features = cols_comunes + ["A_%s"%magnitud]

    #Limpiamos las filas con el dato para esa magnitud a Null
    cols_y_magnitud = cols_features + [magnitud]
    df_datos_magnitud_calculados = df_datos.filter(df_datos["FECHA"] <hoy).select(cols_y_magnitud).na.drop()
    df_datos_magnitud_hoy = df_datos.filter(df_datos["FECHA"] == hoy).select(cols_y_magnitud).na.drop(subset = "A_%s"%magnitud)
    #df_datos_magnitud = df_datos_magnitud_calculados.union(df_datos_magnitud_hoy)
    
    #Assembles to create features column
    assembler = VectorAssembler(inputCols = cols_features, outputCol = "F_%s" % magnitud)
    data_assembled = assembler.transform(df_datos_magnitud_calculados)
    data_to_predict_assembled = assembler.transform(df_datos_magnitud_hoy)
    
    #Seleccionamos las filas que vamos a utilizar
    trainingData = data_assembled.select("CODIGO_CORTO","F_%s" %magnitud, magnitud)
    data_to_predict = data_to_predict_assembled.select("CODIGO_CORTO","F_%s" %magnitud, magnitud)

    # Train a GBT model.
    gbt = GBTRegressor(featuresCol="F_%s" % magnitud, labelCol=magnitud,maxIter=20,predictionCol="P_%s" %magnitud)
        
    # Train model.  This also runs the indexer.
    model = gbt.fit(trainingData)
    
    # Make predictions.
    predictions = model.transform(data_to_predict)
    
    # Select example rows to display
    predictions.select("CODIGO_CORTO","P_%s" %magnitud,"F_%s" % magnitud).show()
    l_predicciones.append(predictions)
    

==================== 1 ====================
+------------+------------------+--------------------+
|CODIGO_CORTO|               P_1|                 F_1|
+------------+------------------+--------------------+
|          40|4.8703098231914215|[40.0,2020.0,5.0,...|
|          57| 6.765132652777576|[57.0,2020.0,5.0,...|
|          17| 8.637713830298447|[17.0,2020.0,5.0,...|
|          35|  4.85415252513568|[35.0,2020.0,5.0,...|
|           4| 3.738079545282313|[4.0,2020.0,5.0,2...|
|           8| 7.742169657289294|[8.0,2020.0,5.0,2...|
|          38|3.7000217826145874|[38.0,2020.0,5.0,...|
|          24| 4.797830391385541|[24.0,2020.0,5.0,...|
|          18| 6.605883601214146|[18.0,2020.0,5.0,...|
|          36|  4.85415252513568|[36.0,2020.0,5.0,...|
+------------+------------------+--------------------+

==================== 10 ====================
+------------+------------------+--------------------+
|CODIGO_CORTO|              P_10|                F_10|
+------------+----------------

+------------+------------------+--------------------+
|CODIGO_CORTO|               P_8|                 F_8|
+------------+------------------+--------------------+
|          27|25.496060655420255|[27.0,2020.0,5.0,...|
|          47|22.064097943230745|[47.0,2020.0,5.0,...|
|          16| 20.74687491039172|[16.0,2020.0,5.0,...|
|          40|22.064097943230745|[40.0,2020.0,5.0,...|
|          57|13.817720794144426|[57.0,2020.0,5.0,...|
|          54|27.919845128821052|[54.0,2020.0,5.0,...|
|          48|21.571929584602085|[48.0,2020.0,5.0,...|
|          17|27.206517742993157|[17.0,2020.0,5.0,...|
|          35|23.121331897489995|[35.0,2020.0,5.0,...|
|           4| 27.86940514698945|[4.0,2020.0,5.0,2...|
|          55| 23.59155619415189|[55.0,2020.0,5.0,...|
|          59|10.314890137886747|[59.0,2020.0,5.0,...|
|           8|31.033059408926754|[8.0,2020.0,5.0,2...|
|          39|27.919845128821052|[39.0,2020.0,5.0,...|
|          49|13.500646829111773|[49.0,2020.0,5.0,...|
|         

### [2.1.0] - Unimos las predicciones por magnitud

In [25]:
df_prediccion = l_predicciones[0]

In [26]:
for i in range(1,len(l_predicciones)):
    df_prediccion = df_prediccion.join(l_predicciones[i],on= "CODIGO_CORTO",how='outer')

In [27]:
pd_prediccion = df_prediccion.toPandas()

# [4] - FORMATO

In [28]:
cols = pd_prediccion.columns.tolist()

In [29]:
regex = reg.compile("P_")
cols_predicciones = [elem for elem in list(filter(regex.search,cols))]

In [30]:
cols = cols[0:1]+cols_predicciones

In [31]:
pd_prediccion = pd_prediccion[cols]

# [5] - EXPORTAR

In [32]:
ayer = (datetime.date.today() + datetime.timedelta(days = -1)).strftime("%Y-%m-%d")
hoy = datetime.date.today().strftime("%Y-%m-%d")

In [33]:
#Versiones
pd_prediccion.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Predicciones/BackUp/Prediccion-" + hoy + ".csv")

In [35]:
pd_prediccion.to_csv("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Predicciones/Prediccion-" + hoy + ".csv")
print("[INFO] - Prediccion-", hoy,".csv --- Generated successfully")

[INFO] - Prediccion- 2020-05-22 .csv --- Generated successfully


In [37]:
#Borrar la de ayer
try:
    os.remove("/home/rulicering/Datos_Proyecto_Ozono/Procesado/Predicciones/Prediccion-" + ayer + ".csv")
    print("[INFO] - Prediccion-", ayer,".csv --- Removed successfully")
except:
    print("[ERROR] - Prediccion-", ayer,".csv --- Could not been removed")

[ERROR] - Prediccion- 2020-05-21 .csv --- Could not been removed
